In [1]:
from utils import load_data
dataset = load_data('../data/dev-v1.1-processed.json')

Load 10570 examples from ../data/dev-v1.1-processed.json...


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data = dataset[55: 85]

In [4]:
def get_word(passage, start, end):
    from preprocess import SpacyTokenizer
    mine = SpacyTokenizer()
    tokenized = [i[0] for i in mine.tokenize(passage).data]
    joined = ' '. join(tokenized[start : end + 1])
    return joined

In [5]:
print('Question:', data[0].question)
print('Answer:', get_word(data[0].passage, data[0].y_start.index(1), data[0].y_end.index(1)))

Question: Who was the Super Bowl 50 MVP ?
Answer: Von Miller


In [6]:
class custom_arguments:
    def __init__(self, args):
        self.batch_size = args['batch_size']
        self.doc_layers = args['doc_layers']
        self.dropout_emb = args['dropout_emb']
        self.dropout_rnn_output = args['dropout_rnn_output']
        self.embedding_file = args['embedding_file']
        self.epoch = args['epoch']
        self.finetune_topk = args['finetune_topk']
        self.gpu = args['gpu']
        self.grad_clipping = args['grad_clipping']
        self.hidden_size = args['hidden_size']
        self.lr = args['lr']
        self.ner_emb_dim = args['ner_emb_dim']
        self.optimizer = args['optimizer']
        self.pos_emb_dim = args['pos_emb_dim']
        self.pretrained = args['pretrained']
        self.rel_emb_dim = args['rel_emb_dim']
        self.rnn_padding = args['rnn_padding']
        self.rnn_type = args['rnn_type']
        self.seed = args['seed']
        self.test_mode = args['test_mode']
        self.use_cuda = args['use_cuda']
        self.p_max_size = args['p_max_size']
        
        
args_dict = {'batch_size':32, 'doc_layers':1, 'dropout_emb':0.4, 'dropout_rnn_output':0.4,
             'embedding_file':'../data/glove.6B.300d.txt',
             'epoch':50, 'finetune_topk':10, 'gpu':'0', 'grad_clipping':10.0, 'hidden_size':96,
             'lr':0.002, 'ner_emb_dim':8, 'optimizer':'adamax', 'pos_emb_dim':12, 'pretrained':'',
             'rel_emb_dim':10, 'rnn_padding':True, 'rnn_type':'lstm', 'seed':1234, 'test_mode':False,
             'use_cuda':True, 'p_max_size':750}
args = custom_arguments(args_dict)

In [7]:
%%capture
import model
from utils import build_vocab
build_vocab()
model = model.Model(args)

In [ ]:
prediction = model.predict(data)
prediction

In [20]:
def output_to_map(prediction):
    maps = []
    for sub_start, sub_end in zip(prediction[0], prediction[1]):
        start = int(torch.argmax(sub_start))
        end = int(torch.argmax(sub_end))
        start, end = min(start, end), max(start, end)
        maps.append([0] * start + [1] * (end - start) + [0]*(len(sub_start) - end))
    return maps

def map_padding(start, end):
    padded = []
    padded.extend([0] * start.index(1))
    padded.extend([1] * (end.index(1) - start.index(1)))
    padded.extend([0] * (len(start) - end.index(1)))
    return padded

In [9]:
evaluate = model.evaluate(data)
evaluate

(tensor(1.00000e-02 *
        [[ 1.5075,  1.4588,  1.4440,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.5075,  1.4588,  1.4440,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.5075,  1.4588,  1.4440,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 1.3652,  1.3211,  1.3077,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.3652,  1.3211,  1.3077,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.3652,  1.3211,  1.3077,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor(1.00000e-02 *
        [[ 1.5304,  1.5360,  1.4690,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.5304,  1.5360,  1.4690,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.5304,  1.5360,  1.4690,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 1.3851,  1.3901,  1.3295,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.3851,  1.3901,  1.3295,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.3851,  1.3901,  1.3295,  ...,  0.0000,  0.0000,  0.0000]]))

In [22]:
import torch
map_pred = output_to_map(evaluate)
for i, data in enumerate(data):
    truth = map_padding(data.y_start, data.y_end)
    f1_scores.append(f1_score(truth, map_pred[i][:len(truth)]))
    a = sum(f1_scores)/len(f1_scores)

TypeError: 'Example' object is not iterable